<a href="https://colab.research.google.com/github/Hyunsyi/test_repo/blob/master/OATutor_Quick_Deploy_%2B_Content_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OATutor Custom Content Quick Deploy
[codebase](https://github.com/CAHLR/OATutor) - [paper](https://doi.org/10.1145/3544548.3581574)



This notebook walks through the necessary terminal commands to process content and deploy OATutor. These commands are run on colab's underlying VM but can be run on any system supporting npm. Alternatively, the platform can be forked and deployed using git-pages (e.g., [https://cahlr.github.io/OATutor](https://cahlr.github.io/OATutor)).

NOTE: A limitation of this colab approach is that the frontend will only run while this notebook session is active.

## Install nodeJS

In [ ]:
!sudo apt install curl
!curl https://raw.githubusercontent.com/creationix/nvm/master/install.sh | bash

## Clone the git repo

In [ ]:
!git clone https://github.com/CAHLR/OATutor.git
!git clone https://github.com/CAHLR/OATutor-Tooling.git
%cd OATutor
# Recursive packages clone flag will require ssh as required in https://github.com/CAHLR/OATutor/blob/main/.gitmodules
# Workaround is to manually set the submodule url to be the https url and then pull from it
!git submodule set-url "src/content-sources/oatutor" https://github.com/CAHLR/OATutor-Content
!git submodule init
!git submodule update

## Process Content

### Install required libraries

In [ ]:
%cd ../OATutor-Tooling/content_script
!pip install -r requirements.txt

### Prepare Excel Content Sheet

In [ ]:
# This cell creates a Content directory under OATutor-Tooling
%cd ..
%mkdir Content

Upload "OpenStax_ College Algebra (Sample).xlsx" to OATutor-Tooling/Content/.

Upload "Problem Bank URL.xlsx" to OATutor-Tooling/Content/.

### Run content script

In [ ]:
%cd content_script

/content/OATutor-Tooling/content_script


In [ ]:
# This converts one content sheet at a time
!python3 process_sheet.py local "../Content/OpenStax_ College Algebra (Sample).xlsx" "1.1 Real Numbers"

At this point, you should see OATutor-Tooling/OpenStax1/ directory, and a list of problem folders inside OpenStax1/. Each problem folder correspond to one problem in 1.1 Real Numbers. Note: you might need to refresh the directory.


In [ ]:
# This converts content from all three sheets
!python3 final.py local "../Content/Problem Bank URL.xlsx"

After the command runs, you should see the JSON content file structure under OATutor-Tooling/OpenStax Content/, as well as `coursePlans.json`, `bktParams.json`, and `skillModel.json` under OATutor-Tooling/. Note: again, you might need to refresh the directory.

### Deploy Locally

Copy content pool and config files from OATutor-Tooling to OATutor.

In [ ]:
%cd /content/OATutor
%ls
# remove old content pool
%rm -rf src/content-sources/oatutor/content-pool

# copy content pool
%cp -a "../OATutor-Tooling/OpenStax Content" src/content-sources/oatutor/content-pool

# copy config files
%cp ../OATutor-Tooling/bktParams.json src/content-sources/oatutor
%cp ../OATutor-Tooling/coursePlans.json src/content-sources/oatutor
%cp ../OATutor-Tooling/skillModel.json src/content-sources/oatutor
%rm src/content-sources/oatutor/bkt-params/*.json
%mv src/content-sources/oatutor/bktParams.json src/content-sources/oatutor/bkt-params/bktParams1.json
%cp src/content-sources/oatutor/bkt-params/bktParams1.json src/content-sources/oatutor/bkt-params/bktParams2.json

# preprocess content
%cd src/tools
!node preprocessProblemPool.js
%cd ../..

## Install OATutor node dependencies

In [ ]:
!npm install

## Deploy

In [ ]:
# This command runs the frontend server in the background
!while true; do npm run start >frontend-log.txt 2>&1; sleep 5; done >/dev/null 2>&1 &

#### The below cell will contain debug logs of the frontend, re-run the cell to fetch the latest logs

In [ ]:
!cat frontend-log.txt

## Set up localtunnel to access Colab web server

In [ ]:
!npm install -g localtunnel

In [ ]:
# This command runs the local tunnel port forwarding in the background
!while true; do nohup lt --port 3001 >url.txt 2>&1; sleep 5; done >/dev/null 2>&1 &

#### Get URL to frontend. Re-run to view latest logs from runtime.

In [ ]:
!cat url.txt

#### Get the IP address of the Colab VM that the above URL will ask for

In [ ]:
!curl ipv4.icanhazip.com